# Data Cleaning
The cleaning process is organized into sections, each corresponding to a different dataset (`.csv` file). Each section includes the following steps:

1. **Data Understanding**: Initial exploration of the dataset.
2. **Data Cleaning**: Handling of missing values (NaN), removal of duplicates, setting correct data types, and renaming columns. <br>
   *(Optional)* **Deep Clean**: Custom cleaning steps applied to a specific dataset, if necessary.
3. **Final Result**: Displays the cleaned dataset and saves it to a new `.csv` file.

All the datasets combined have a total size of ~1Gb and can be all uploaded in memory at the same time on almost every PC.

First, import the necessary libraries and set up any required options.

In [1]:
import pandas as pd
import numpy as np

from utils.utils import find_matching

# Set to True to print cleaned data into new csv
PRINT_CSV = False

## Movies

In [2]:
# Import 'movies.csv' dataset
movies_df = pd.read_csv('datasets/movies.csv')

### 1. Data Understanding

In [3]:
movies_df.head()

,id,name,date,tagline,description,minute,rating
0,1000001,Barbie,2023.0,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114.0,3.86
1,1000002,Parasite,2019.0,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133.0,4.56
2,1000003,Everything Everywhere All at Once,2022.0,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140.0,4.30
3,1000004,Fight Club,1999.0,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139.0,4.27
4,1000005,La La Land,2016.0,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129.0,4.09


In [4]:
movies_df.shape

(941597, 7)

In [5]:
movies_df.dtypes

id               int64
name            object
date           float64
tagline         object
description     object
minute         float64
rating         float64
dtype: object

### 2. Data Cleaning

In [6]:
# Rename columns
movies_df = movies_df.rename(columns={'name': 'title', 'minute': 'duration_in_minutes', 'date': 'release_year'})
print(f"Movies dataset columns: {', '.join(movies_df.columns)}")

Movies dataset columns: id, title, release_year, tagline, description, duration_in_minutes, rating


In [7]:
# Check for null values
movies_df.isna().sum()

id                          0
title                      10
release_year            91913
tagline                802210
description            160812
duration_in_minutes    181570
rating                 850598
dtype: int64

There are null values in most of the columns.
The fields '**release_year**', '**tagline**', '**description**', '**duration_in_minutes**' and '**rating**' don't cause any problems so we'll keep them, but the few movies that are without a title can't be used and will be removed.

In [8]:
# Removing rows with null title
no_title = movies_df[movies_df['title'].isna()]
movies_df = movies_df.dropna(subset=['title'])

print("Movies dataset without title:")
no_title.head()

Movies dataset without title:


,id,title,release_year,tagline,description,duration_in_minutes,rating
287514,1287515,NaN,2015.0,NaN,NONE is a short film that explores the balance...,4.0,NaN
617642,1617643,NaN,NaN,NaN,NaN,NaN,NaN
646520,1646521,NaN,2008.0,NaN,NaN,NaN,NaN
648185,1648186,NaN,NaN,NaN,NaN,NaN,NaN
720294,1720295,NaN,NaN,NaN,"In this directorial debut of Eden Ewardson, he...",8.0,NaN


In [9]:
# Check for duplicate rows
print(f"There are {movies_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [10]:
# Setting the correct type for columns
movies_df['release_year'] = movies_df['release_year'].astype('Int64')
movies_df['duration_in_minutes'] = movies_df['duration_in_minutes'].astype('Int64')
movies_df[['release_year', 'duration_in_minutes']].dtypes

release_year           Int64
duration_in_minutes    Int64
dtype: object

In [11]:
# Check if 'id' column has unique values
print(f"'id' duplicates: {movies_df[movies_df['id'].duplicated()].shape[0]}")
movies_df = movies_df.set_index("id")

'id' duplicates: 0


The '**id**' field is the unique identifier of a movie, so it's been set as the index.

#### Deep Clean
Let's look inside some columns to see most frequent values:

In [12]:
movies_df['description'].value_counts().head(10)

description
Mexican feature film    893
Plot Unavailable.       504
Hong Kong movie         449
to be added later       304
Chinese movie           203
plot is unknown         182
Short film.             180
A short film.           168
1962 Japanese movie     162
Documentary film.       151
Name: count, dtype: int64

Many descriptions seem to have a description like "Plot Unavailable" or similar instead of a null value. The other fields seem fine.<br>
Let's try to fix as many as possible (fixing only the most frequent variation, not 100% accurate).

In [13]:
from utils.utils import null_movie_description_keywords

# Find null description variation
matches = find_matching(movies_df, 'description', null_movie_description_keywords, max_length=30)

# Fill with NaN values the result obtained
movies_df.loc[matches.index, ['description']] = np.nan

# Manual check to be sure to not overwrite real descriptions
matches['description'].value_counts().head(15)

description
Plot Unavailable.         504
plot is unknown           182
Plot Unknown              103
Plot Unknown.              45
Plot unknown.              35
Plot unknown               26
Plot details unknown.      16
Synopsis unknown.          11
Plot is unknown             9
plot is unknown.            5
plot unknown                5
plot currently unknown      2
Plot is Unknown             2
Overview unknown.           2
Plot unavailable            2
Name: count, dtype: int64

### 3. Final Result
All datasets reference the **movies** dataset. A movie is uniquely identified by his **id** and a movie id has multiple occurrences in other datasets. A movie has a title, a tagline, a description, the release year, the duration and a rating. Only the title is mandatory and all the other attributes could be missing.

In [14]:
movies_df.head()

,title,release_year,tagline,description,duration_in_minutes,rating
id,,,,,,
1000001,Barbie,2023,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114,3.86
1000002,Parasite,2019,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133,4.56
1000003,Everything Everywhere All at Once,2022,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140,4.30
1000004,Fight Club,1999,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139,4.27
1000005,La La Land,2016,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129,4.09


In [15]:
movies_df.shape

(941587, 6)

## Languages

In [16]:
# Import 'languages.csv' dataset
lang_df = pd.read_csv('datasets/languages.csv')

### 1. Data Understanding

In [17]:
lang_df.head()

,id,type,language
0,1000001,Language,English
1,1000002,Primary language,Korean
2,1000002,Spoken language,English
3,1000002,Spoken language,German
4,1000002,Spoken language,Korean


In [18]:
lang_df.shape

(1038762, 3)

In [19]:
lang_df.dtypes

id           int64
type        object
language    object
dtype: object

### 2. Data Cleaning

In [20]:
# Rename columns
lang_df = lang_df.rename(columns={'id': 'movie_id'})
print(f"Languages dataset columns: {', '.join(lang_df.columns)}")

Languages dataset columns: movie_id, type, language


In [21]:
# Check for null values
lang_df.isna().sum()

movie_id    0
type        0
language    0
dtype: int64

In [22]:
# Check for duplicate rows
print(f"There are {lang_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [23]:
# Setting the category data type for column 'type'
print(f"types: {', '.join(lang_df['type'].unique())}")
lang_df['type'] = lang_df['type'].astype('category')

types: Language, Primary language, Spoken language


The `type` field has only 3 possible values, so we can set it as a categorical type.

### 3. Final Result
The languages dataset is directly linked to the movies dataset through the `movie_id` column. There are more rows in the languages dataset than in the movies dataset because a movie can be associated with multiple languages. Additionally, not all movies have a language defined. <br>
A language associated with a movie can fall into one or more of the following categories:
- *Language*: Refers to a generic language associated with the movie, typically used when there is a single dominant language.
- *Primary Language*: The main or original language of the movie.
- *Spoken Language*: All the languages actually used in the movie's dialogues.

In [24]:
lang_df.head()

,movie_id,type,language
0,1000001,Language,English
1,1000002,Primary language,Korean
2,1000002,Spoken language,English
3,1000002,Spoken language,German
4,1000002,Spoken language,Korean


In [25]:
lang_df.shape

(1038762, 3)

## Actors

In [26]:
# Import 'actors.csv' dataset
actors_df = pd.read_csv('datasets/actors.csv')

### 1. Data Understanding

In [27]:
actors_df.head()

,id,name,role
0,1000001,Margot Robbie,Barbie
1,1000001,Ryan Gosling,Ken
2,1000001,America Ferrera,Gloria
3,1000001,Ariana Greenblatt,Sasha
4,1000001,Issa Rae,Barbie


In [28]:
actors_df.shape

(5798450, 3)

In [29]:
actors_df.dtypes

id       int64
name    object
role    object
dtype: object

### 2. Data Cleaning

In [30]:
# Rename columns
actors_df = actors_df.rename(columns={'id': 'movie_id'})
print(f"Actors dataset columns: {', '.join(actors_df.columns)}")

Actors dataset columns: movie_id, name, role


In [31]:
# Check for null values
actors_df.isna().sum()

movie_id          0
name              4
role        1361559
dtype: int64

Despite the high number of null values in the `role` field, the rows will be maintained because they still contain information about the actor's `name`. However, an actor without a name is unusable, so the corresponding rows will be removed.

In [32]:
# Removing actors without name
no_name = actors_df[actors_df['name'].isna()]
actors_df = actors_df.dropna(subset=['name'])
no_name

,movie_id,name,role
4145738,1443629,NaN,NaN
4281100,1469981,NaN,Self
4306960,1474958,NaN,Cinematography
5430275,1773264,NaN,NaN


In [33]:
# Check for duplicate rows
print(f"There are {actors_df.duplicated().sum()} duplicated rows")
actors_duplicates = actors_df[actors_df.duplicated(keep=False)].head(6)

# Dropping the duplicates
actors_df = actors_df.drop_duplicates()

actors_duplicates

There are 946 duplicated rows


,movie_id,name,role
3967,1000062,Rosie Jones,Lady of the Boot of Jemiah
3993,1000062,Rosie Jones,Lady of the Boot of Jemiah
44615,1000797,Karel Heřmánek,Czech Injured Man
44642,1000797,Karel Heřmánek,Czech Injured Man
47806,1000863,Michael Fennimore,Car Salesman
47807,1000863,Michael Fennimore,Car Salesman


Completely duplicated rows are clearly errors and can be removed.

#### Deep Clean

In [34]:
actors_df['role'].value_counts().head(10)

role
Self                      188466
Himself                    74797
Herself                    22678
Self (archive footage)     21851
Narrator                   12276
(uncredited)               10816
(voice)                     8435
Narrator (voice)            7569
Dancer                      6874
Doctor                      5742
Name: count, dtype: int64

The role column contains many variations of the 'Self' role. Let's examine this more closely.

In [35]:
from utils.utils import self_actor_role_keywords

# Find self variation
matches = find_matching(actors_df, 'role', self_actor_role_keywords)
print(f"Rows contains 'self' variations: {matches['role'].shape[0]}")
matches['role'].value_counts().head()

Rows contains 'self' variations: 375241


role
Self                      188466
Himself                    74797
Herself                    22678
Self (archive footage)     21851
himself                     5273
Name: count, dtype: int64

There are over 300,000 values similar to 'Self', but many of them also contain additional information, such as 'Self - Presenter' or 'Self - Guest'. Overwriting all these values could lead to a loss of information, so they won't be overwritten in the cleaned dataset. However, they may be modified when visualizing the data for statistical purposes.

In [36]:
# Reset indexing after removing rows
actors_df = actors_df.reset_index(drop=True)

### 3. Final Result
The actors dataset is directly linked to the movies dataset through the `movie_id` column and contains nearly six times the number of rows as the movies dataset. Additionally, a movie may have no actors associated with it. <br>
The same actor can appear multiple times in the dataset if they feature in more than one movie. <br>
An actor is identified solely by their full name, stored in a single field.


In [37]:
actors_df.head()

,movie_id,name,role
0,1000001,Margot Robbie,Barbie
1,1000001,Ryan Gosling,Ken
2,1000001,America Ferrera,Gloria
3,1000001,Ariana Greenblatt,Sasha
4,1000001,Issa Rae,Barbie


In [38]:
actors_df.shape

(5797500, 3)

## Countries

In [39]:
# Import 'countries.csv' dataset
countries_df = pd.read_csv('datasets/countries.csv')

### 1. Data Understanding

In [40]:
countries_df.head()

,id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany


In [41]:
countries_df.shape

(693476, 2)

In [42]:
countries_df.dtypes

id          int64
country    object
dtype: object

### 2. Data Cleaning

In [43]:
# Rename columns
countries_df = countries_df.rename(columns={'id': 'movie_id'})
print(f"Countries dataset columns: {', '.join(countries_df.columns)}")

Countries dataset columns: movie_id, country


In [44]:
# Check for null values
countries_df.isna().sum()

movie_id    0
country     0
dtype: int64

In [45]:
# Check for duplicate rows
print(f"There are {countries_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


### 3. Final Results
The **countries** dataset is directly connected to the movies dataset through the 'movie_id' column. This dataset contains all the countries where the movies were produced.


In [46]:
countries_df.head()

,movie_id,country
0,1000001,UK
1,1000001,USA
2,1000002,South Korea
3,1000003,USA
4,1000004,Germany


In [47]:
countries_df.shape

(693476, 2)

## Crew

In [48]:
# Import 'crew.csv' dataset
crew_df = pd.read_csv('datasets/crew.csv')

### 1. Data Understanding

In [49]:
crew_df.head()

,id,role,name
0,1000001,Director,Greta Gerwig
1,1000001,Producer,Tom Ackerley
2,1000001,Producer,Margot Robbie
3,1000001,Producer,Robbie Brenner
4,1000001,Producer,David Heyman


In [50]:
crew_df.shape

(4720183, 3)

In [51]:
crew_df.dtypes

id       int64
role    object
name    object
dtype: object

### 2. Data Cleaning

In [52]:
# Rename columns
crew_df = crew_df.rename(columns={'id': 'movie_id', 'name': 'crew_member_name'})
print(f"Crew dataset columns: {', '.join(crew_df.columns)}")

Crew dataset columns: movie_id, role, crew_member_name


In [53]:
# Check for null values
crew_df.isna().sum()

movie_id            0
role                0
crew_member_name    1
dtype: int64

In [54]:
find_matching(crew_df, 'crew_member_name', ["anonymous", "unknown"]).head()

,movie_id,role,crew_member_name
122552,1001960,Co-director,Anonymous
488047,1014226,Executive producer,Anonymous Anonymous
510633,1015341,Director,Anonymous
510634,1015341,Editor,Anonymous
740024,1028791,Original writer,Anonymous


The existing `NaN` value was not removed because it is minimal compared to the overall size of the dataset, and removing it would not significantly impact the analysis. Similarly, values such as *Unknown* or *Anonymous* were kept because they account for less than 0.001% of the data and do not affect the overall results.

In [55]:
# Check for duplicate rows
print('Duplicated rows:', crew_df.duplicated().sum())
crew_duplicates = crew_df[crew_df.duplicated(keep=False)].head()

# Dropping the duplicates
crew_df = crew_df.drop_duplicates()

crew_duplicates

Duplicated rows: 1282


,movie_id,role,crew_member_name
1680,1000018,Stunts,Chris Webb
1721,1000018,Stunts,Chris Webb
2690,1000031,Stunts,Sarah Irwin
2691,1000031,Stunts,Sarah Irwin
2692,1000031,Stunts,Sarah Irwin


Completely duplicated rows are clearly an error and can be safely removed.

In [56]:
# Reset indexing after removing rows
crew_df = crew_df.reset_index(drop=True)

### 3. Final Results
The **crew** dataset is connected to the movies dataset through the `movie_id` column. It includes the names of all crew members along with their roles. <br>
A crew member can have different roles in the same movies and can appear in more than one movie. <br>
A crew member is solely identified by his full name.

In [57]:
crew_df.head()

,movie_id,role,crew_member_name
0,1000001,Director,Greta Gerwig
1,1000001,Producer,Tom Ackerley
2,1000001,Producer,Margot Robbie
3,1000001,Producer,Robbie Brenner
4,1000001,Producer,David Heyman


In [58]:
crew_df.shape

(4718901, 3)

## Genres

In [59]:
# Import 'genres.csv' dataset
genres_df = pd.read_csv('datasets/genres.csv')

### 1. Data Understanding

In [60]:
genres_df.head()

,id,genre
0,1000001,Comedy
1,1000001,Adventure
2,1000002,Comedy
3,1000002,Thriller
4,1000002,Drama


In [61]:
genres_df.shape

(1046849, 2)

In [62]:
genres_df.dtypes

id        int64
genre    object
dtype: object

### 2. Data Cleaning

In [63]:
# Rename columns
genres_df = genres_df.rename(columns={'id': 'movie_id'})
print(f"Genres dataset columns: {', '.join(genres_df.columns)}")

Genres dataset columns: movie_id, genre


In [64]:
# Check for null values
genres_df.isna().sum()

movie_id    0
genre       0
dtype: int64

In [65]:
# Check for duplicate rows
print(f"There are {genres_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [66]:
# Setting the correct type for columns
genres_list = list(genres_df["genre"].unique())
print(f'There are {len(genres_list)} genres in the dataset: {", ".join(genres_list)}')

genres_df['genre'] = genres_df['genre'].astype('category')

There are 19 genres in the dataset: Comedy, Adventure, Thriller, Drama, Science Fiction, Action, Music, Romance, History, Crime, Animation, Mystery, Horror, Family, Fantasy, War, Western, TV Movie, Documentary


Given the limited number of genres in the dataset, the field can be optimized setting it to category type.

### 3. Final Results
The **genres** dataset is connected to the movies dataset through the `movie_id` column. A movie can have multiple genres.


In [67]:
genres_df.head()

,movie_id,genre
0,1000001,Comedy
1,1000001,Adventure
2,1000002,Comedy
3,1000002,Thriller
4,1000002,Drama


In [68]:
genres_df.shape

(1046849, 2)

## Posters

In [69]:
# Import 'posters.csv' dataset
posters_df = pd.read_csv('datasets/posters.csv')

### 1. Data Understanding

In [70]:
posters_df.head()

,id,link
0,1000001,https://a.ltrbxd.com/resized/film-poster/2/7/7...
1,1000002,https://a.ltrbxd.com/resized/film-poster/4/2/6...
2,1000003,https://a.ltrbxd.com/resized/film-poster/4/7/4...
3,1000004,https://a.ltrbxd.com/resized/film-poster/5/1/5...
4,1000005,https://a.ltrbxd.com/resized/film-poster/2/4/0...


In [71]:
posters_df.shape

(941597, 2)

In [72]:
posters_df.dtypes

id       int64
link    object
dtype: object

### 2. Data Cleaning

In [73]:
# Rename columns
posters_df = posters_df.rename(columns={'id': 'movie_id', 'link': 'poster_link'})
print(f"Posters dataset columns: {', '.join(posters_df.columns)}")

Posters dataset columns: movie_id, poster_link


In [74]:
# Check for null values
posters_df.isna().sum()

movie_id            0
poster_link    180712
dtype: int64

In [75]:
# Removing null rows
posters_df = posters_df.dropna()

Removing the `NaN` values as they do not contribute meaningful information to the dataset and could hinder data consistency and analysis. <br>
We also need to check the validity of the link:

In [76]:
# Check for invalid links
link_regex = r'\bhttps?:\/\/(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?:\/[^\s]*)?'
print(f"There are {(~posters_df['poster_link'].str.contains(link_regex, na=False)).sum()} invalid links.")

There are 0 invalid links.


In [77]:
# Check for duplicate rows
print(f"There are {posters_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [78]:
# Check if a movie can have more than 1 poster
print(f"There are {posters_df['movie_id'].duplicated().sum()} duplicates in the 'movie_id' column.")

There are 0 duplicates in the 'movie_id' column.


A movie can have at most only one poster. The relationship between the **posters** dataset and the **movies** dataset is One-to-One, allowing us to merge the two datasets.

In [79]:
# Merging the datasets on 'id' from 'movies' and 'movie_id' from 'posters'
movies_df = pd.merge(movies_df, posters_df, left_on='id', right_on='movie_id', how='left').drop(columns=['movie_id'])

### 3. Final Results
The posters dataset has now been merged into the movies dataset under the `poster_link` column. To ensure the merge was successful, the dataset should have the same number of rows as before (previously 941,587) and one additional column (previously 6).

In [80]:
movies_df.head()

,title,release_year,tagline,description,duration_in_minutes,rating,poster_link
0,Barbie,2023,She's everything. He's just Ken.,Barbie and Ken are having the time of their li...,114,3.86,https://a.ltrbxd.com/resized/film-poster/2/7/7...
1,Parasite,2019,Act like you own the place.,"All unemployed, Ki-taek's family takes peculia...",133,4.56,https://a.ltrbxd.com/resized/film-poster/4/2/6...
2,Everything Everywhere All at Once,2022,The universe is so much bigger than you realize.,An aging Chinese immigrant is swept up in an i...,140,4.30,https://a.ltrbxd.com/resized/film-poster/4/7/4...
3,Fight Club,1999,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,139,4.27,https://a.ltrbxd.com/resized/film-poster/5/1/5...
4,La La Land,2016,Here's to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...",129,4.09,https://a.ltrbxd.com/resized/film-poster/2/4/0...


In [81]:
movies_df.shape

(941587, 7)

## Releases

In [82]:
# Import 'release.csv' dataset
releases_df = pd.read_csv('datasets/releases.csv')

### 1. Data Understanding

In [83]:
releases_df.head()

,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN


In [84]:
releases_df.shape

(1332782, 5)

In [85]:
releases_df.dtypes

id          int64
country    object
date       object
type       object
rating     object
dtype: object

### 2. Data Cleaning

In [86]:
# Rename columns
releases_df = releases_df.rename(columns={'id': 'movie_id', 'type': 'distribution_format'})
print(f"Release dataset columns: {', '.join(releases_df.columns)}")

Release dataset columns: movie_id, country, date, distribution_format, rating


In [87]:
# Check for null values
releases_df.isna().sum()

movie_id                    0
country                     0
date                        0
distribution_format         0
rating                 998802
dtype: int64

It's fine for the dataset to have null values  in the `rating` column. <br>
The absence of a rating could also be due to some countries not having an official rating system. If that is the case, we don't have enough information to determine whether it's a missing value or a country without a rating system. To make the data more meaningful, we should identify the countries without an official rating system and differentiate them from with a specific value. However, for our purposes, we don't need to clean the data to that extent.

In [88]:
# Check for alternative null values in the dataset
releases_df[(releases_df['rating'] == "0") & (~releases_df['country'].isin(["Germany", "Austria", "Switzerland"]))].head()

,movie_id,country,date,distribution_format,rating
216945,1009817,Hungary,2013-08-22,Theatrical,0
223492,1010561,Thailand,2011-11-21,Physical,0
279945,1018726,Israel,2022-06-16,Digital,0
289168,1020593,Israel,2022-06-16,Digital,0
291982,1021166,Israel,2022-06-16,Digital,0


We initially checked the rating value 0 in the dataset, assuming it might represent a null or missing value. However, we discovered that in some countries (e.g., Germany), a rating of 0 has a meaningful interpretation, indicating that the film is suitable for all audiences, including children. While in some cases the 0 rating might still be an error, it appears only 70 times in a dataset of over one million rows. Given that we don't have a reference dataset with the rating system for every country, we can ignore these values.

In [89]:
# Check for duplicate rows
print(f"There are {releases_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [90]:
# Setting the correct type for the columns
releases_df['date'] = pd.to_datetime(releases_df['date'], format='%Y-%m-%d')
releases_df['distribution_format'] = releases_df['distribution_format'].astype('category')

distribution_formats = list(releases_df['distribution_format'].unique())
print(f'There are {len(distribution_formats)} distribution formats in the dataset: {", ".join(distribution_formats)}')

There are 6 distribution formats in the dataset: Theatrical, Digital, Physical, Premiere, Theatrical limited, TV


Given the limited number of distribution formats in the dataset, the field can be optimized setting it to category type.

### 3. Final Results
The **release** dataset is linked to the movies dataset through the 'movie_id' column. It contains details about the movie releases worldwide, including the release date, distribution format, and rating.

In [91]:
releases_df.head()

,movie_id,country,date,distribution_format,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN


In [92]:
releases_df.shape

(1332782, 5)

## Studios

In [93]:
# Import 'studios.csv' dataset
studios_df = pd.read_csv('datasets/studios.csv')

### 1. Data Understanding

In [94]:
studios_df.head()

,id,studio
0,1000001,LuckyChap Entertainment
1,1000001,Heyday Films
2,1000001,NB/GG Pictures
3,1000001,Mattel
4,1000001,Warner Bros. Pictures


In [95]:
studios_df.shape

(679283, 2)

In [96]:
studios_df.dtypes

id         int64
studio    object
dtype: object

### 2. Data Cleaning

In [97]:
# Rename columns
studios_df = studios_df.rename(columns={'id': 'movie_id'})
print(f"Studios dataset columns: {', '.join(studios_df.columns)}")

Studios dataset columns: movie_id, studio


In [98]:
# Check for null values
studios_df.isna().sum()

movie_id     0
studio      10
dtype: int64

Unnamed studios are not usable and can be safely removed.

In [99]:
# Removing rows with null studio name
no_name_studios = studios_df[studios_df['studio'].isna()]
studios_df = studios_df.dropna(subset=['studio'])

print("Studios without name:")
no_name_studios.head()

Studios without name:


,movie_id,studio
347347,1259717,NaN
411467,1350206,NaN
473794,1450762,NaN
534117,1565428,NaN
541076,1579904,NaN


In [100]:
# Check for duplicate rows
print(f"There are {studios_df.duplicated().sum()} duplicated rows")
studios_duplicates = studios_df[studios_df.duplicated(keep=False)].head(6)

# Drop duplicates
studios_df = studios_df.drop_duplicates()

studios_duplicates

There are 212 duplicated rows


,movie_id,studio
145,1000044,Working Title Films
146,1000044,Working Title Films
485,1000165,Working Title Films
487,1000165,Working Title Films
809,1000263,Working Title Films
810,1000263,Working Title Films


Completely duplicated rows are clearly an error and can be safely removed.

In [101]:
# Reset indexing after removing rows
studios_df = studios_df.reset_index(drop=True)

### 3. Final Results
The **studios** dataset is linked to the movies dataset through the `movie_id` column. It lists all the studios involved in each movie, allowing a movie to be associated with multiple studios and a studio to collaborate on multiple movies.

In [102]:
studios_df.head()

,movie_id,studio
0,1000001,LuckyChap Entertainment
1,1000001,Heyday Films
2,1000001,NB/GG Pictures
3,1000001,Mattel
4,1000001,Warner Bros. Pictures


In [103]:
studios_df.shape

(679061, 2)

## Themes

In [104]:
# Import 'themes.csv' dataset
themes_df = pd.read_csv('datasets/themes.csv')

### 1. Data Understanding

In [105]:
themes_df.head()

,id,theme
0,1000001,Humanity and the world around us
1,1000001,Crude humor and satire
2,1000001,Moving relationship stories
3,1000001,Emotional and captivating fantasy storytelling
4,1000001,Surreal and thought-provoking visions of life ...


In [106]:
themes_df.shape

(125641, 2)

In [107]:
themes_df.dtypes

id        int64
theme    object
dtype: object

### 2. Data Cleaning

In [108]:
# Rename columns
themes_df = themes_df.rename(columns={'id': 'movie_id'})
print(f"Themes dataset columns: {', '.join(themes_df.columns)}")

Themes dataset columns: movie_id, theme


In [109]:
# Check for null values
themes_df.isna().sum()

movie_id    0
theme       0
dtype: int64

In [110]:
# Check for duplicate rows
print(f"There are {themes_df.duplicated().sum()} duplicated rows")

There are 0 duplicated rows


In [111]:
# Check for unique values
print(f"There are {len(themes_df['theme'].unique())} unique themes")

# Setting the correct type for columns
themes_df['theme'] = themes_df['theme'].astype('category')

There are 109 unique themes


There are a limited number of values for the theme, which means the column can be set as a categorical variable to optimize the analysis.

### 3. Final Results
The **themes** dataset is linked to the movies dataset through the `movie_id` column. It contains 125,000 rows, with multiple occurrences referring to the same movie, suggesting that themes are not frequently assigned to movies. Each theme describes a movie using few standard phrases.

In [112]:
themes_df.head()

,movie_id,theme
0,1000001,Humanity and the world around us
1,1000001,Crude humor and satire
2,1000001,Moving relationship stories
3,1000001,Emotional and captivating fantasy storytelling
4,1000001,Surreal and thought-provoking visions of life ...


In [113]:
themes_df.shape

(125641, 2)

## The Oscar Awards

In [114]:
# Import 'the_oscar_awards.csv' dataset
oscars_df = pd.read_csv('datasets/the_oscar_awards.csv')

### 1. Data Understanding

In [115]:
oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [116]:
oscars_df.shape

(10889, 7)

In [117]:
oscars_df.dtypes

year_film         int64
year_ceremony     int64
ceremony          int64
category         object
name             object
film             object
winner             bool
dtype: object

### 2. Data Cleaning

In [118]:
# Rename columns
oscars_df = oscars_df.rename(columns={'name': 'nominee_name', 'film': 'nominated_film', 'winner': 'is_winner'})
print(f"Oscar dataset columns: {', '.join(oscars_df.columns)}")

Oscar dataset columns: year_film, year_ceremony, ceremony, category, nominee_name, nominated_film, is_winner


In [119]:
# Check for null values
oscars_df.isna().sum()

year_film           0
year_ceremony       0
ceremony            0
category            0
nominee_name        5
nominated_film    319
is_winner           0
dtype: int64

In [120]:
# Check for duplicate rows
print(f"There are {oscars_df.duplicated().sum()} duplicated rows")
oscars_df[oscars_df.duplicated(keep=False)].head(6)

There are 7 duplicated rows


,year_film,year_ceremony,ceremony,category,nominee_name,nominated_film,is_winner
6219,1983,1984,56,MUSIC (Original Song),Music by Michel Legrand; Lyric by Alan Bergman...,Yentl,False
6220,1983,1984,56,MUSIC (Original Song),Music by Michel Legrand; Lyric by Alan Bergman...,Yentl,False
7066,1991,1992,64,MUSIC (Original Song),Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,False
7068,1991,1992,64,MUSIC (Original Song),Music by Alan Menken; Lyric by Howard Ashman,Beauty and the Beast,False
7394,1994,1995,67,MUSIC (Original Song),Music by Elton John; Lyric by Tim Rice,The Lion King,False
7395,1994,1995,67,MUSIC (Original Song),Music by Elton John; Lyric by Tim Rice,The Lion King,False


Duplicates are retained because, in music categories, the same artists can receive identical nominations for different songs, with the song titles not specified in the dataset.

In [121]:
# Check the consistency between year_film and year_ceremony
print(f"There are {oscars_df[oscars_df['year_film'] > oscars_df['year_ceremony']].shape[0]} rows where the movie has been released after ceremony")

There are 0 rows where the movie has been released after ceremony


In [122]:
# Check for multiple winner possibilities
from utils.utils import special_oscar_awards

# Filtering rows where the 'category' is in the special_oscar_awards list
filtered_oscars = find_matching(oscars_df, 'category', special_oscar_awards)

# Find groups with more than one winner
multiple_winners = filtered_oscars.groupby(['year_ceremony', 'category']).filter(
    lambda x: x['is_winner'].sum() > 1
)

# Keep only the rows where 'is_winner' is True
multiple_winners = multiple_winners[multiple_winners['is_winner'] == True]
multiple_winners.head(6)

,year_film,year_ceremony,ceremony,category,nominee_name,nominated_film,is_winner
33,1927,1928,1,SPECIAL AWARD,Warner Bros.,NaN,True
34,1927,1928,1,SPECIAL AWARD,Charles Chaplin,NaN,True
520,1936,1937,9,SPECIAL AWARD,The March of Time for its significance to mot...,NaN,True
521,1936,1937,9,SPECIAL AWARD,W. Howard Greene and Harold Rosson for the co...,NaN,True
640,1937,1938,10,SPECIAL AWARD,Mack Sennett,NaN,True
641,1937,1938,10,SPECIAL AWARD,Edgar Bergen for his outstanding comedy creation,NaN,True


There can be multiple winners for the same Oscar category (e.g., 1932 Best Actor). <br>
In the count, we excluded special awards, such as the Jean Hersholt Humanitarian Award, because they are given without nominations and have an undefined number of winners each year. These awards are not part of traditional competitive categories but rather honorary recognitions.

In [123]:
# Check for unique values
print(f"There are {len(oscars_df['category'].unique())} unique categories")

# Setting the correct type for columns
oscars_df['category'] = oscars_df['category'].astype('category')

There are 115 unique categories


There are a limited number of values for the `category` field, which means the column can be set as a categorical variable to optimize the analysis.

### 3. Final Results

In [124]:
oscars_df.head()

,year_film,year_ceremony,ceremony,category,nominee_name,nominated_film,is_winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [125]:
oscars_df.shape

(10889, 7)

## Rotten Tomatoes Reviews

In [126]:
# Import 'rotten_tomatoes_reviews' dataset
reviews_df = pd.read_csv('datasets/rotten_tomatoes_reviews.csv')

### 1. Data Understanding

In [127]:
reviews_df.head()

,rotten_tomatoes_link,movie_title,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,Percy Jackson & the Olympians: The Lightning T...,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [128]:
reviews_df.shape

(1129887, 9)

In [129]:
reviews_df.dtypes

rotten_tomatoes_link    object
movie_title             object
critic_name             object
top_critic                bool
publisher_name          object
review_type             object
review_score            object
review_date             object
review_content          object
dtype: object

### 2. Data Cleaning

In [130]:
# Rename columns
reviews_df = reviews_df.rename(columns={'review_type': 'type', 'review_score': 'score', 'review_content': 'content', 'top_critic': 'is_top_critic'})
print(f"Rotten Tomatoes Reviews dataset columns: {', '.join(reviews_df.columns)}")

Rotten Tomatoes Reviews dataset columns: rotten_tomatoes_link, movie_title, critic_name, is_top_critic, publisher_name, type, score, review_date, content


In [131]:
# Check for null values
reviews_df.isna().sum()

rotten_tomatoes_link         0
movie_title                  0
critic_name              18521
is_top_critic                0
publisher_name               0
type                         0
score                   305902
review_date                  0
content                  65778
dtype: int64

After checking the Rotten Tomatoes website, it's fine to have the publisher name, critic name, and content as null values.

In [132]:
# Check for duplicate rows
print(f"There are {reviews_df.duplicated().sum()} duplicated rows")

filtered_df = reviews_df[reviews_df['critic_name'].notna()]
filtered_df = filtered_df[filtered_df.duplicated(keep=False)]

reviews_df = reviews_df.drop(reviews_df[reviews_df['critic_name'].isna()].index)

filtered_df.head()

There are 119471 duplicated rows


,rotten_tomatoes_link,movie_title,critic_name,is_top_critic,publisher_name,type,score,review_date,content
35513,m/1069696-screamers,Screamers,Dave White,False,Movies.com,Fresh,B-,1996-01-26,NaN
35514,m/1069696-screamers,Screamers,Dave White,False,Movies.com,Fresh,B-,1996-01-26,NaN
35576,m/1069707-othello,Othello,Fred Topel,False,About.com,Fresh,4/5,2003-11-25,Fine Shakespeare adaptation
35577,m/1069707-othello,Othello,Fred Topel,False,About.com,Fresh,4/5,2003-11-25,Fine Shakespeare adaptation
135520,m/animatrix,The Animatrix,Michael Dequina,False,TheMovieReport.com,Fresh,3/5,2007-01-12,NaN


There are many duplicate reviews in the datasets. After checking the official website, it is possible to have multiple reviews for the same movie from the same publisher and with an unspecified critic. These rows will be excluded from the total count of duplicates and will not be removed. All other duplicate rows are being removed.

In [133]:
# Check for unique values
print(f"There are {len(reviews_df['type'].unique())} review types")

# Setting the correct type for columns
reviews_df['type'] = reviews_df['type'].astype('category')
reviews_df['review_date'] = pd.to_datetime(reviews_df['review_date'], format='%Y-%m-%d')

There are 2 review types


A review can be either Fresh or Rotten, so the `type` column is set as a categorical variable.

In [134]:
# Reset indexing after removing rows
reviews_df = reviews_df.reset_index(drop=True)

### 3. Final Result

In [135]:
reviews_df.head()

,rotten_tomatoes_link,movie_title,critic_name,is_top_critic,publisher_name,type,score,review_date,content
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
3,m/0814255,Percy Jackson & the Olympians: The Lightning T...,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
4,m/0814255,Percy Jackson & the Olympians: The Lightning T...,David Germain,True,Associated Press,Rotten,NaN,2010-02-10,It's more a list of ingredients than a movie-m...


In [136]:
reviews_df.shape

(1111366, 9)

---
## Save the cleaned dataset to new `.csv` files

In [137]:
# Print the clean datasets to new csv files
if PRINT_CSV:
    movies_df.to_csv('clean_datasets/movies.csv')
    lang_df.to_csv('clean_datasets/languages.csv')
    actors_df.to_csv('clean_datasets/actors.csv')
    countries_df.to_csv('clean_datasets/countries.csv')
    crew_df.to_csv('clean_datasets/crew.csv')
    genres_df.to_csv('clean_datasets/genres.csv')
    releases_df.to_csv('clean_datasets/releases.csv')
    studios_df.to_csv('clean_datasets/studios.csv')
    themes_df.to_csv('clean_datasets/themes.csv')
    oscars_df.to_csv('clean_datasets/oscars.csv')
    reviews_df.to_csv('clean_datasets/reviews.csv')